# DISH enrichment tables

## Import modules

In [1]:
# Import modules
import pandas as pd
import numpy as np
from datetime import datetime

%matplotlib inline
%load_ext autoreload
%autoreload 2

## Define functions

In [55]:
def open_file(my_file_path):
    '''
    Function:   
        + Opens a csv file and converts it into a pandas dataframe with all fields read as string (except NaN)
    Input:
        + my_file_path  => Path of the file
    Output:
        + my_file_as_df => Pandas dataframe
    '''

    # Variable control
    assert isinstance(my_file_path, str), 'The path to the file must be as string'
    assert len(my_file_path) > 0, 'Path is empty'

    # Code
    with open(my_file_path) as my_file: # Opening this way it is not necessary to close the file to release resources
        my_file_as_df = pd.read_csv(my_file, dtype=str)
    
    return my_file_as_df

In [56]:
def export_to_xlsx(path, df):
    with pd.ExcelWriter(path) as writer:
        df.to_excel(writer, sheet_name='py_export', index=False)

In [57]:
def zeros_to_add(val,):
    if val == 10: return 0
    if val >= 9: return 1
    if val >= 8: return 2
    if val >= 7: return 3
    if val >= 6: return 4
    if val >= 5: return 5
    if val >= 4: return 6
    if val >= 3: return 7
    if val >= 2: return 8
    if val >= 1: return 9

In [58]:
def padding(NR_Cell_ID_dec, NR_Cell_ID_dec_len):
    if NR_Cell_ID_dec_len == 10: return '313340' + NR_Cell_ID_dec
    elif NR_Cell_ID_dec_len == 9: return '313340' + '0' + NR_Cell_ID_dec
    elif NR_Cell_ID_dec_len == 8: return '313340' + '00' + NR_Cell_ID_dec
    elif NR_Cell_ID_dec_len == 7: return '313340' + '000' + NR_Cell_ID_dec
    elif NR_Cell_ID_dec_len == 6: return '313340' + '0000' + NR_Cell_ID_dec
    elif NR_Cell_ID_dec_len == 5: return '313340' + '00000' + NR_Cell_ID_dec
    elif NR_Cell_ID_dec_len == 4: return '313340' + '000000' + NR_Cell_ID_dec
    elif NR_Cell_ID_dec_len == 3: return '313340' + '0000000' + NR_Cell_ID_dec
    elif NR_Cell_ID_dec_len == 2: return '313340' + '00000000' + NR_Cell_ID_dec
    elif NR_Cell_ID_dec_len == 1: return '313340' + '000000000' + NR_Cell_ID_dec
    else: return 'error'

## Load files

In [ ]:
# Especify the path to the file
path_to_n1_acdm = 'C:/Users/alvaro.mendoza/Desktop/N1 ACDM Export/20220825113059.csv'
path_to_n1_rfciq = 'C:/Users/alvaro.mendoza/Desktop/N1 RF CIQ_Table/20220825121623.csv'

In [59]:
df_n1_acdm = open_file(path_to_n1_acdm)
df_n1_rfciq = open_file(path_to_n1_rfciq)

n1_acdm_col = ['NR Cell ID', 'Cell Site Common Name', 'Cell Name'] # 20220825113059
n1_rfciq_col = ['NR Cell ID','Site ID', 'TX ID'] # 20220825121623

df_n1_acdm = df_n1_acdm[n1_acdm_col]
df_n1_rfciq = df_n1_rfciq[n1_rfciq_col]

df_n1_acdm = df_n1_acdm.sort_values(by=['Cell Site Common Name', 'Cell Name'], ignore_index=True)
df_n1_rfciq = df_n1_rfciq.sort_values(by=['Site ID', 'TX ID'], ignore_index=True)

df_n1_acdm = df_n1_acdm.dropna(subset=['NR Cell ID'])
df_n1_rfciq = df_n1_rfciq.dropna(subset=['NR Cell ID'])

df_n1_acdm = df_n1_acdm.drop_duplicates(subset=['NR Cell ID'], keep='first')
df_n1_rfciq = df_n1_rfciq.drop_duplicates(subset=['NR Cell ID'], keep='first')

df_n1_acdm['NR Cell ID'] = df_n1_acdm['NR Cell ID'].astype(str)
df_n1_rfciq['NR Cell ID'] = df_n1_rfciq['NR Cell ID'].astype(str)

In [60]:
df_n1_acdm.head()

,NR Cell ID,Cell Site Common Name,Cell Name
0,04BEDD003,ALALB00003A,ALALB00003A_n29_E_DL_1
1,04BEDD004,ALALB00003A,ALALB00003A_n29_E_DL_2
2,04BEDD005,ALALB00003A,ALALB00003A_n29_E_DL_3
3,04BEDD009,ALALB00003A,ALALB00003A_n66_AWS-4_DL_1
4,04BEDD00A,ALALB00003A,ALALB00003A_n66_AWS-4_DL_2


In [64]:
#old approach
#df_n1_acdm['NR Cell ID len'] = df_n1_acdm['NR Cell ID'].apply(len)
#df_n1_acdm = df_n1_acdm[df_n1_acdm['NR Cell ID len'] == 9]
#df_n1_acdm['NR Cell ID'] = df_n1_acdm['NR Cell ID'].str.lower()
#df_n1_acdm['KEY'] = df_n1_acdm['Cell Name'] + '_' + df_n1_acdm['NR Cell ID']
#df_n1_acdm = df_n1_acdm.drop_duplicates(subset=['KEY'], keep='first')

#df_n1_acdm['NR CGI hex'] = '0x133304' + df_n1_acdm['NR Cell ID']
#df_n1_acdm['NR CGI hex'] = df_n1_acdm['NR CGI hex'].astype(str)
#df_n1_acdm['NR CGI dec'] = df_n1_acdm['NR CGI hex'].apply(int, base=16)
#df_n1_acdm

#new approach
df_n1_acdm['NR Cell ID len'] = df_n1_acdm['NR Cell ID'].apply(len)
df_n1_acdm['NR Cell ID'] = df_n1_acdm['NR Cell ID'].str.lower()
df_n1_acdm['NR Cell ID'] = '0x' + df_n1_acdm['NR Cell ID']
df_n1_acdm['NR Cell ID dec'] = df_n1_acdm['NR Cell ID'].apply(int, base=16)
df_n1_acdm['NR Cell ID dec'] = df_n1_acdm['NR Cell ID dec'].astype(str)
df_n1_acdm['NR Cell ID dec len'] = df_n1_acdm['NR Cell ID dec'].apply(len)
df_n1_acdm

,NR Cell ID,Cell Site Common Name,Cell Name,NR Cell ID len,NR Cell ID dec,NR Cell ID dec len
0,0x04bedd003,ALALB00003A,ALALB00003A_n29_E_DL_1,9,1273876483,10
1,0x04bedd004,ALALB00003A,ALALB00003A_n29_E_DL_2,9,1273876484,10
2,0x04bedd005,ALALB00003A,ALALB00003A_n29_E_DL_3,9,1273876485,10
3,0x04bedd009,ALALB00003A,ALALB00003A_n66_AWS-4_DL_1,9,1273876489,10
4,0x04bedd00a,ALALB00003A,ALALB00003A_n66_AWS-4_DL_2,9,1273876490,10
...,...,...,...,...,...,...
250224,0x069b8711e,SYSYR00553A,SYSYR00553A_n70_AWS-4_UL15_2,9,1773695262,10
250225,0x069b8711f,SYSYR00553A,SYSYR00553A_n70_AWS-4_UL15_3,9,1773695263,10
250226,0x069b87117,SYSYR00553A,SYSYR00553A_n71_F-G_1,9,1773695255,10
250227,0x069b87118,SYSYR00553A,SYSYR00553A_n71_F-G_2,9,1773695256,10


In [65]:
df_n1_acdm["NR Cell ID len"].unique()

array([ 9, 10], dtype=int64)

In [66]:
df_n1_acdm["NR Cell ID dec len"].unique()

array([10,  9], dtype=int64)

In [67]:
# let suppouse should be mcc+mnc+cell_id => 3+3+10=16
# => Drop any cell_id greather than 10
# => For cell_id lenght less than 10, add 0 to complete 10

In [68]:
df_n1_acdm['NR CGI dec'] = df_n1_acdm.apply(lambda x: padding(x['NR Cell ID dec'], x['NR Cell ID dec len']), axis = 1)
df_n1_acdm

,NR Cell ID,Cell Site Common Name,Cell Name,NR Cell ID len,NR Cell ID dec,NR Cell ID dec len,NR CGI dec
0,0x04bedd003,ALALB00003A,ALALB00003A_n29_E_DL_1,9,1273876483,10,3133401273876483
1,0x04bedd004,ALALB00003A,ALALB00003A_n29_E_DL_2,9,1273876484,10,3133401273876484
2,0x04bedd005,ALALB00003A,ALALB00003A_n29_E_DL_3,9,1273876485,10,3133401273876485
3,0x04bedd009,ALALB00003A,ALALB00003A_n66_AWS-4_DL_1,9,1273876489,10,3133401273876489
4,0x04bedd00a,ALALB00003A,ALALB00003A_n66_AWS-4_DL_2,9,1273876490,10,3133401273876490
...,...,...,...,...,...,...,...
250224,0x069b8711e,SYSYR00553A,SYSYR00553A_n70_AWS-4_UL15_2,9,1773695262,10,3133401773695262
250225,0x069b8711f,SYSYR00553A,SYSYR00553A_n70_AWS-4_UL15_3,9,1773695263,10,3133401773695263
250226,0x069b87117,SYSYR00553A,SYSYR00553A_n71_F-G_1,9,1773695255,10,3133401773695255
250227,0x069b87118,SYSYR00553A,SYSYR00553A_n71_F-G_2,9,1773695256,10,3133401773695256


In [69]:
# Set name and path for the exported file
today_date = datetime.today().strftime('%Y-%m-%d')
today_date = today_date.replace('-', '')

file_name_acdm = today_date + ' - 20220825113059_v2' + '.xlsx'
export_path_acdm = 'C:/Users/alvaro.mendoza/Desktop/N1 ACDM Export/'
file_exp_path_acdm = export_path_acdm + file_name_acdm

file_name_rfciq = today_date + ' - 20220825121623_v2' + '.xlsx'
export_path_rfciq = 'C:/Users/alvaro.mendoza/Desktop/N1 RF CIQ_Table/'
file_exp_path_rfciq = export_path_rfciq + file_name_rfciq


In [70]:
export_to_xlsx(file_exp_path_acdm, df_n1_acdm)

In [146]:
export_to_xlsx(file_exp_path_rfciq, df_n1_rfciq)

In [ ]:
# TX ID and Cell Name will be the KEY
1 drop all samples without NR_CellID
3059 => 206975 samples
1623 => 303136 samples
2 drop all unused columns
3 hex2dec the NR_CellID if needed
4 columns homogenization (cell_name and site_name)
5 create the nRCGI as Martin said
5 create new key => cell_name+NR_CellID_dec

In [ ]:
What I saw
1) the files have different cell_name (some of them are the same, but just few)
2) there are NR_CellID missing => we should drop them as they are part of the nRCGI
3) we can generate the nRCGI just with the NR_CellID (mnc and mcc we already know)
4) then we can "combine" the two files and remove duplicates and do the vlookup


In [ ]:
20220825121623
Steps
delete all columns except Site ID, TX ID (actully Cell Name) and NR Cell ID
Dish Network is MCC = 313 + MNC = 340 allways


20220825113059
Steps
delete all columns except Site ID, Cell Name) and NR Cell ID
Dish Network is MCC = 313 + MNC = 340 allways

re-arrange both of them

